In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-books-dataset/Amazon_Books_Scraping/Genre_df.csv
/kaggle/input/amazon-books-dataset/Amazon_Books_Scraping/Books_df.csv
/kaggle/input/amazon-books-dataset/Amazon_Books_Scraping/Sub_Genre_df.csv


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
     

In [3]:
books = pd.read_csv('/kaggle/input/amazon-books-dataset/Amazon_Books_Scraping/Books_df.csv')
genre = pd.read_csv('/kaggle/input/amazon-books-dataset/Amazon_Books_Scraping/Genre_df.csv')
sub_genre = pd.read_csv('/kaggle/input/amazon-books-dataset/Amazon_Books_Scraping/Sub_Genre_df.csv')

In [4]:
books.drop(['Unnamed: 0'], axis=1,inplace = True)

In [5]:
genre.columns

Index(['Title', 'Number of Sub-genres', 'URL'], dtype='object')

In [6]:
# Join df1 and df2 on different columns
merged_df = pd.merge(books, genre, left_on='Main Genre', right_on='Title', how='inner')
# merged_df.head(10)
# # # Join the resulting DataFrame with df3 on another column
final_df = pd.merge(merged_df, sub_genre, left_on='Sub Genre', right_on='Title', how='inner')


In [7]:
final_df.head(10)

,Title_x,Author,Main Genre_x,Sub Genre,Type,Price,Rating,No. of People rated,URLs_x,Title_y,Number of Sub-genres,URL,Title,Main Genre_y,No. of Books,URLs_y
0,The Complete Novel of Sherlock Holmes,Arthur Conan Doyle,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹169.00,4.4,19923.0,https://www.amazon.in/Complete-Novels-Sherlock-Holmes/dp/8175994312/ref=zg_bs_g_1318054031_d_sccl_1/000-0000000-0000000?psc=1,"Arts, Film & Photography",11,https://www.amazon.in/gp/bestsellers/books/1318052031,Cinema & Broadcast,"Arts, Film & Photography",100.0,https://www.amazon.in/gp/bestsellers/books/1318054031
1,"Black Holes (L) : The Reith Lectures [Paperback] Hawking, Stephen",Stephen Hawking,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹99.00,4.5,7686.0,https://www.amazon.in/Black-Holes-Lectures-Stephen-Hawking/dp/085750357X/ref=zg_bs_g_1318054031_d_sccl_2/000-0000000-0000000?psc=1,"Arts, Film & Photography",11,https://www.amazon.in/gp/bestsellers/books/1318052031,Cinema & Broadcast,"Arts, Film & Photography",100.0,https://www.amazon.in/gp/bestsellers/books/1318054031
2,The Kite Runner,Khaled Hosseini,"Arts, Film & Photography",Cinema & Broadcast,Kindle Edition,₹175.75,4.6,50016.0,https://www.amazon.in/Kite-Runner-Khaled-Hosseini-ebook/dp/B00B0CR0O6/ref=zg_bs_g_1318054031_d_sccl_3/000-0000000-0000000?psc=1,"Arts, Film & Photography",11,https://www.amazon.in/gp/bestsellers/books/1318052031,Cinema & Broadcast,"Arts, Film & Photography",100.0,https://www.amazon.in/gp/bestsellers/books/1318054031
3,Greenlights: Raucous stories and outlaw wisdom from the Academy Award-winning actor,Matthew McConaughey,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹389.00,4.6,32040.0,https://www.amazon.in/Greenlights-Raucous-stories-Academy-Award-winning/dp/1472280873/ref=zg_bs_g_1318054031_d_sccl_4/000-0000000-0000000?psc=1,"Arts, Film & Photography",11,https://www.amazon.in/gp/bestsellers/books/1318052031,Cinema & Broadcast,"Arts, Film & Photography",100.0,https://www.amazon.in/gp/bestsellers/books/1318054031
4,"The Science of Storytelling: Why Stories Make Us Human, and How to Tell Them Better",Will Storr,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹348.16,4.5,1707.0,https://www.amazon.in/Science-Storytelling-Will-Storr/dp/0008276978/ref=zg_bs_g_1318054031_d_sccl_5/000-0000000-0000000?psc=1,"Arts, Film & Photography",11,https://www.amazon.in/gp/bestsellers/books/1318052031,Cinema & Broadcast,"Arts, Film & Photography",100.0,https://www.amazon.in/gp/bestsellers/books/1318054031
5,Fantastic Beasts and Where to Find Them: The Original Screenplay,J.K. Rowling,"Arts, Film & Photography",Cinema & Broadcast,Kindle Edition,₹313.95,4.5,15122.0,https://www.amazon.in/Fantastic-Beasts-Where-Find-Them-ebook/dp/B01ETJABRE/ref=zg_bs_g_1318054031_d_sccl_6/000-0000000-0000000?psc=1,"Arts, Film & Photography",11,https://www.amazon.in/gp/bestsellers/books/1318052031,Cinema & Broadcast,"Arts, Film & Photography",100.0,https://www.amazon.in/gp/bestsellers/books/1318054031
6,Fantastic Beasts: The Crimes of Grindelwald - The Original Screenplay,J.K. Rowling,"Arts, Film & Photography",Cinema & Broadcast,Kindle Edition,₹313.95,4.6,10152.0,https://www.amazon.in/Fantastic-Beasts-Grindelwald-Original-Screenplay-ebook/dp/B07BL2VNNV/ref=zg_bs_g_1318054031_d_sccl_7/000-0000000-0000000?psc=1,"Arts, Film & Photography",11,https://www.amazon.in/gp/bestsellers/books/1318052031,Cinema & Broadcast,"Arts, Film & Photography",100.0,https://www.amazon.in/gp/bestsellers/books/1318054031
7,Desperately Seeking Shah Rukh : India's Lonely Young Women and the Search for Intimacy and Independence,Shrayana Bhattacharya,"Arts, Film & Photography",Cinema & Broadcast,Paperback,₹348.16,4.5,539.0,https://www.amazon.in/Desperately-Seeking-Shah-Rukh-Independence/dp/9356292140/ref=zg_bs_g_1318054031_d_sccl_8/000-0000000-0000000?psc=1,"Arts, Film & Photography",11,https://www.amazon.in/gp/bestsellers/books/1318052031,Cinema & Broadcast,"Arts, Film & Photography",100.0,https://w